Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. Create a convolutional neural network that trains to 100% accuracy on these images, which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import zipfile
from os import path, getcwd, chdir

# DO NOT CHANGE THE LINE BELOW. If you are developing in a local
# environment, then grab happy-or-sad.zip from the Coursera Jupyter Notebook
# and place it inside a local folder and edit the path to that location
path = "/content/drive/MyDrive/study/tf-specialization/course1_intro_to_ai_ml_dl/week 4/tmp2/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [15]:
class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if(logs.get("accuracy") >= 0.999):
      print("Reached 99% training accuracy so stopping")
      self.model.stop_training = True

In [3]:
# data dir
train_dir = "/tmp/h-or-s"

In [10]:
# get data and load it
from keras.preprocessing import image

train_datagen = image.ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size = (150,150),
    class_mode = "binary"
)



Found 80 images belonging to 2 classes.


In [18]:
# create model

model = keras.models.Sequential([
    keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(150,150,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128, (3,3), activation="relu"),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(256, (3,3), activation="relu"),
    keras.layers.MaxPooling2D(2,2),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 72, 72, 128)       73856     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 36, 36, 128)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 34, 34, 256)       295168    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 17, 17, 256)       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 73984)            

In [20]:
# compile

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.RMSprop(lr=0.001), metrics=["accuracy"])

In [21]:
callbacks = myCallback()
model.fit_generator(
    train_generator,
    epochs = 20,
    callbacks=[callbacks]
)

Epoch 1/20
3/3 [==============================] - 6s 2s/step - loss: 19.6820 - accuracy: 0.5500
Epoch 2/20
3/3 [==============================] - 6s 2s/step - loss: 0.9508 - accuracy: 0.4750
Epoch 3/20
3/3 [==============================] - 6s 2s/step - loss: 0.6609 - accuracy: 0.6750
Epoch 4/20
3/3 [==============================] - 6s 2s/step - loss: 0.5557 - accuracy: 0.6500
Epoch 5/20
3/3 [==============================] - 8s 3s/step - loss: 0.7896 - accuracy: 0.7625
Epoch 6/20
3/3 [==============================] - 6s 2s/step - loss: 0.4020 - accuracy: 0.8000
Epoch 7/20
3/3 [==============================] - 7s 2s/step - loss: 0.2381 - accuracy: 0.9375
Epoch 8/20
3/3 [==============================] - 6s 2s/step - loss: 0.2166 - accuracy: 0.9250
Epoch 9/20
3/3 [==============================] - 6s 2s/step - loss: 0.2440 - accuracy: 0.8250
Epoch 10/20
3/3 [==============================] - 6s 2s/step - loss: 0.1344 - accuracy: 0.9375
Epoch 11/20
3/3 [==============================]

In [22]:
model.save("ex4.h5")